#### Load Packages

In [149]:
#loads in packages 
import scipy
import sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from collections import Counter

#### Read datasets (all jobs and manually data)

In [150]:
all_jobs = pd.read_csv('all_jobs_cleaned.csv') #import all_jobs_cleaned

test_dataframe = pd.read_csv('testing_data.csv') #import all_jobs_cleaned
test_dataframe

,job_title_clean,company,date,blurb,tags,link,id,job level,remoteness,state,city,wage_type,converted_yearly_lower,converted_yearly_upper
0,senior data engineer,walmart,11/7/2024,performs initial data quality checks on extrac...,"Benefits: Paid jury duty, Paid parental leave,...",https://www.indeed.com/m/viewjob?jk=dd69111a39...,IndeedScraperUSAEng_dd69111a392c05c4,Senior-Level,unknown,ca,sunnyvale,yearly,117000.0,234000.0
1,store scanning specialist,retaildata llc,10/28/2024,flexible part time jobs now available in your ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=4a21ddda73...,IndeedScraperUSAEng_4a21ddda7395ed1a,Entry-Level,unknown,nc,apex,NaN,NaN,NaN
2,marketing applied scientist ii,uber,10/3/2024,experience with big data using technologies li...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=61ac412466...,IndeedScraperUSAEng_61ac4124667518a5,Mid-Level,unknown,ny,new york,yearly,149000.0,165500.0
3,research data analyst hybrid opportunity year...,stanford university,10/3/2024,analyze data processes in documentation. the a...,Job types: Full-time\r\nBenefits: Tuition reim...,https://www.indeed.com/m/viewjob?jk=bbb6a310c0...,IndeedScraperUSAEng_bbb6a310c0afcc13,Entry-Level,unknown,ca,stanford,yearly,82468.0,106256.0
4,bioinformatics data engineer,revolution medicines,10/5/2024,carry out data modeling and curation to ensure...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=a4d2ecf9b7...,IndeedScraperUSAEng_a4d2ecf9b7c35fa9,Entry-Level,unknown,ca,redwood city,yearly,120000.0,145000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,director data impact,nyc office of the mayor,10/3/2024,"indicators, data sharing and open-data standar...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=b47e13a030...,IndeedScraperUSAEng_b47e13a030427fe1,Mid-Level,unknown,ny,new york,yearly,103000.0,120000.0
595,senior director data analytics availability ...,salesforce,11/2/2024,"to get the best candidate experience, please c...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=f15c91a342...,IndeedScraperUSAEng_f15c91a3428aeac5,Senior-Level,unknown,ca,san francisco,yearly,20400.0,372900.0
596,principal data scientist,intuit,10/3/2024,drive focused efforts on end-to-end data quali...,NaN,https://www.indeed.com/m/viewjob?jk=82e0a2aa71...,IndeedScraperUSAEng_82e0a2aa71e55cf8,Senior-Level,unknown,ca,mountain view,NaN,NaN,NaN
597,data engineer ecommerce,tiktok,10/5/2024,"experience with performing data analysis, data...","Benefits: Paid parental leave, Health savings ...",https://www.indeed.com/m/viewjob?jk=5e43120afd...,IndeedScraperUSAEng_5e43120afd8547ba,Mid-Level,unknown,ca,san jose,yearly,136000.0,280000.0


In [151]:
test_dataframe['title-blurb'] = test_dataframe['job_title_clean'] + ' ' + test_dataframe['blurb']
test_dataframe = test_dataframe[['id', 'title-blurb','job level']]
test_dataframe['job level'].value_counts()
test_dataframe

,id,title-blurb,job level
0,IndeedScraperUSAEng_dd69111a392c05c4,senior data engineer performs initial data qua...,Senior-Level
1,IndeedScraperUSAEng_4a21ddda7395ed1a,store scanning specialist flexible part time j...,Entry-Level
2,IndeedScraperUSAEng_61ac4124667518a5,marketing applied scientist ii experience with...,Mid-Level
3,IndeedScraperUSAEng_bbb6a310c0afcc13,research data analyst hybrid opportunity year...,Entry-Level
4,IndeedScraperUSAEng_a4d2ecf9b7c35fa9,bioinformatics data engineer carry out data mo...,Entry-Level
...,...,...,...
594,IndeedScraperUSAEng_b47e13a030427fe1,"director data impact indicators, data sharing...",Mid-Level
595,IndeedScraperUSAEng_f15c91a3428aeac5,senior director data analytics availability ...,Senior-Level
596,IndeedScraperUSAEng_82e0a2aa71e55cf8,principal data scientist drive focused efforts...,Senior-Level
597,IndeedScraperUSAEng_5e43120afd8547ba,data engineer ecommerce experience with perfor...,Mid-Level


#### Remove NaN values from job title

In [152]:
all_jobs[all_jobs['job_title_clean'].isna()]
all_jobs.dropna(subset=['job_title_clean'], inplace=True)
#test_dataframe.dropna(subset=['job_title_clean'], inplace=True)

all_jobs.head()

,job_title_clean,company,date,blurb,tags,link,id,remoteness,state,city,wage_type,converted_yearly_lower,converted_yearly_upper
0,direct data entry operator,westamerica bank,2024-10-17,"as a direct data entry operator, under general...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=9af5925791...,IndeedScraperUSAEng_9af5925791536fd2,unknown,ca,fairfield,hourly,33280.0,35505.6
1,cloud data architectdirector,cedent,2024-10-03,"provide data engineers, quant engineers and re...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=66827a8f8e...,IndeedScraperUSAEng_66827a8f8e70c0e9,unknown,ca,san francisco,NaN,NaN,NaN
2,data aide,stanford university,2024-10-03,develop and apply novel clinical data curation...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=db7d847fbf...,IndeedScraperUSAEng_db7d847fbf8a9612,unknown,ca,stanford,hourly,64916.8,74048.0
3,field applications engineer data center,amphenol tcs,2024-10-22,amphenol high speed and commercial products gr...,"Benefits: Paid holidays, Disability insurance,...",https://www.indeed.com/m/viewjob?jk=cfb3f4f072...,IndeedScraperUSAEng_cfb3f4f072371529,unknown,ca,santa clara,yearly,80750.0,148870.0
4,analyst youthtruth,cep global,2024-10-03,"over time, contribute insights to new features...","Benefits: Dental insurance, 401(k), Flexible s...",https://www.indeed.com/m/viewjob?jk=9365df687b...,IndeedScraperUSAEng_9365df687b5694e9,unknown,ca,san francisco,yearly,90756.0,90756.0


#### Remove test data from training dataset

In [153]:
#separate training and testing
training_dataset = all_jobs[~all_jobs['id'].isin(test_dataframe['id'])]
training_dataset

,job_title_clean,company,date,blurb,tags,link,id,remoteness,state,city,wage_type,converted_yearly_lower,converted_yearly_upper
0,direct data entry operator,westamerica bank,2024-10-17,"as a direct data entry operator, under general...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=9af5925791...,IndeedScraperUSAEng_9af5925791536fd2,unknown,ca,fairfield,hourly,33280.0,35505.6
1,cloud data architectdirector,cedent,2024-10-03,"provide data engineers, quant engineers and re...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=66827a8f8e...,IndeedScraperUSAEng_66827a8f8e70c0e9,unknown,ca,san francisco,NaN,NaN,NaN
2,data aide,stanford university,2024-10-03,develop and apply novel clinical data curation...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=db7d847fbf...,IndeedScraperUSAEng_db7d847fbf8a9612,unknown,ca,stanford,hourly,64916.8,74048.0
3,field applications engineer data center,amphenol tcs,2024-10-22,amphenol high speed and commercial products gr...,"Benefits: Paid holidays, Disability insurance,...",https://www.indeed.com/m/viewjob?jk=cfb3f4f072...,IndeedScraperUSAEng_cfb3f4f072371529,unknown,ca,santa clara,yearly,80750.0,148870.0
4,analyst youthtruth,cep global,2024-10-03,"over time, contribute insights to new features...","Benefits: Dental insurance, 401(k), Flexible s...",https://www.indeed.com/m/viewjob?jk=9365df687b...,IndeedScraperUSAEng_9365df687b5694e9,unknown,ca,san francisco,yearly,90756.0,90756.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10599,independent scanning specialist,retaildata llc,2024-11-26,flexible part time jobs now available in your ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=dad664e0f7...,IndeedScraperUSAEng_dad664e0f79e7b2d,unknown,nc,durham,NaN,NaN,NaN
10600,software engineer aiml interface development,bae systems,2024-11-27,"knowledge of data structures, algorithms, and ...",Job types: Full-time\r\nRemote: Hybrid work\r\...,https://www.indeed.com/m/viewjob?jk=551fcfa4fc...,IndeedScraperUSAEng_551fcfa4fc547502,temporarily_remote,nc,durham,yearly,92290.0,156860.0
10601,pricing integrity agent,retaildata llc,2024-11-26,"we have part-time, flexible work schedules ava...",Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=f2424b8e30...,IndeedScraperUSAEng_f2424b8e30eb6199,unknown,nc,chapel hill,NaN,NaN,NaN
10602,senior it business analyst telematicscrm,"stefanini, inc",2024-11-23,"experience in application data modeling, activ...",Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=19c64cca4c...,IndeedScraperUSAEng_19c64cca4c42b3e3,unknown,nc,greensboro,hourly,114400.0,126880.0


In [154]:
#randomly select 1400 values from the dataset -- you want this subset to be used for the chatgpt model 
chatgpt_rows = training_dataset.sample(n=1400) #this is your training dataset 
chatgpt_rows

,job_title_clean,company,date,blurb,tags,link,id,remoteness,state,city,wage_type,converted_yearly_lower,converted_yearly_upper
6605,principal full stack engineer database graphql...,fidelity investments,2024-10-26,exploring new technology trends and using them...,NaN,https://www.indeed.com/m/viewjob?jk=4f2d54d10d...,IndeedScraperUSAEng_4f2d54d10d4bf1ad,unknown,tx,westlake,NaN,NaN,NaN
9115,director i data science,liberty mutual,2024-11-22,subject areas our team tackles include complex...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=35daf34116...,IndeedScraperUSAEng_35daf34116a49738,unknown,tx,plano,yearly,133200.0,246900.0
6641,analyst data management,ally financial,2024-10-03,experience and working knowledge in extracting...,Job types: Full-time\r\nShifts: 8 hour shift\r...,https://www.indeed.com/m/viewjob?jk=8f54296560...,IndeedScraperUSAEng_8f54296560cf25b4,temporarily_remote,tx,lewisville,yearly,55000.0,95000.0
4917,data reporting specialist,hra/dept of social services,2024-10-09,aggregate and clean vendor management data fro...,Job types: Full-time\r\nBenefits: Loan forgive...,https://www.indeed.com/m/viewjob?jk=06ebaa320c...,IndeedScraperUSAEng_06ebaa320caf4e24,unknown,ny,manhattan,yearly,95070.0,109330.0
2311,data quality engineer java or c,s&p global,2024-11-14,"perform queries to identify, dirty data, orpha...","Benefits: Health insurance, Referral program",https://www.indeed.com/m/viewjob?jk=40e053ff0d...,IndeedScraperUSAEng_40e053ff0d791c1a,unknown,ca,monterey,yearly,80000.0,100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,store scanning specialist,retaildata llc,2024-10-29,field representative - flexible schedules for ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=2cf671cb12...,IndeedScraperUSAEng_2cf671cb12cdf3fa,unknown,nc,chapel hill,NaN,NaN,NaN
2856,data scanning associate,retaildata llc,2024-11-19,field representative - flexible schedules for ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=07d1ef688b...,IndeedScraperUSAEng_07d1ef688b32265d,unknown,ca,oakland,NaN,NaN,NaN
9539,data scientist,mag aerospace,2024-10-05,presenting information using data visualizatio...,Job types: Full-time\r\nBenefits: Health insur...,https://www.indeed.com/m/viewjob?jk=6fedecb074...,IndeedScraperUSAEng_6fedecb074edd327,unknown,nc,fort liberty,yearly,63750.0,106250.0
8897,pricing specialist,republic national distributing company,2024-11-10,perform data maintenance: conduct routine data...,"Benefits: Paid training, Paid parental leave, ...",https://www.indeed.com/m/viewjob?jk=e013ee922a...,IndeedScraperUSAEng_e013ee922a7d9926,unknown,tx,grand prairie,NaN,NaN,NaN


In [155]:
# #label your training data using chatgpt code
entry_keywords = [' i ','staff', 'dba', 'junior', 'assistant', 'entry', 'graduate', 'intern', 'trainee', 'aide','collector','technician' ]
mid_keywords = [' ii','mid', 'associate', 'specialist', 'coordinator', 'analyst', 'engineer', 'developer']
senior_keywords = ['mentor', 'supervisor', 'sr. ','head ','sr ', 'president', ' iii','senior', 'lead', 'manager', 'director', 'principal', 'executive', 'chief']

# Function to label job levels based on the title and blurb
def label_job_level(row):
    title = str(row['job_title_clean']).lower()
    blurb = str(row['blurb']).lower()

    if any(keyword in title or keyword in blurb for keyword in senior_keywords):
        return 'Senior-Level'
    elif any(keyword in title or keyword in blurb for keyword in mid_keywords):
        return 'Mid-Level'
    elif any(keyword in title or keyword in blurb for keyword in entry_keywords):
        return 'Entry-Level'
    else:
        return 'Uncategorized'

# Apply the labeling function
chatgpt_rows['job_level'] = chatgpt_rows.apply(label_job_level, axis=1)

# Count job levels for an overview
job_level_counts = chatgpt_rows['job_level'].value_counts()
job_level_counts

Senior-Level     626
Mid-Level        536
Uncategorized    137
Entry-Level      101
Name: job_level, dtype: int64

In [156]:
# import numpy as np

# def classify_job_level(row):
#     title = row['job_title_clean'].lower()
#     blurb = row['blurb'].lower() if isinstance(row['blurb'], str) else ""
#     salary_lower = row['converted_yearly_lower']
    
#     # Entry Level Conditions
#     if any(term in title for term in ['assistant', 'coordinator', 'entry', 'intern', 'operator']) or \
#        any(term in blurb for term in ['training provided', 'no experience required']) or \
#        (pd.notna(salary_lower) and salary_lower < 60000):
#         return "Entry Level"
    
#     # Senior Level Conditions
#     if any(term in title for term in ['director', 'manager', 'senior', 'architect']) or \
#        any(term in blurb for term in ['leadership', 'advanced expertise', 'team lead']) or \
#        (pd.notna(salary_lower) and salary_lower > 100000):
#         return "Senior Level"
    
#     # Mid Level as Default
#     return "Mid Level"

# # Apply the classification function to each row
# chatgpt_rows['job_level'] = chatgpt_rows.apply(classify_job_level, axis=1)

# # Display updated dataset with the new column
# chatgpt_rows[['job_title_clean', 'company', 'converted_yearly_lower', 'converted_yearly_upper', 'job_level']].head()

In [157]:
#vectorize the job title column
#first combine title and blurb together in 1 column:
chatgpt_rows['title-blurb'] = chatgpt_rows['job_title_clean'] + ' ' + chatgpt_rows['blurb']
chatgpt_rows

,job_title_clean,company,date,blurb,tags,link,id,remoteness,state,city,wage_type,converted_yearly_lower,converted_yearly_upper,job_level,title-blurb
6605,principal full stack engineer database graphql...,fidelity investments,2024-10-26,exploring new technology trends and using them...,NaN,https://www.indeed.com/m/viewjob?jk=4f2d54d10d...,IndeedScraperUSAEng_4f2d54d10d4bf1ad,unknown,tx,westlake,NaN,NaN,NaN,Senior-Level,principal full stack engineer database graphql...
9115,director i data science,liberty mutual,2024-11-22,subject areas our team tackles include complex...,Job types: Full-time\r\nJob types cc: Full-time,https://www.indeed.com/m/viewjob?jk=35daf34116...,IndeedScraperUSAEng_35daf34116a49738,unknown,tx,plano,yearly,133200.0,246900.0,Senior-Level,director i data science subject areas our team...
6641,analyst data management,ally financial,2024-10-03,experience and working knowledge in extracting...,Job types: Full-time\r\nShifts: 8 hour shift\r...,https://www.indeed.com/m/viewjob?jk=8f54296560...,IndeedScraperUSAEng_8f54296560cf25b4,temporarily_remote,tx,lewisville,yearly,55000.0,95000.0,Senior-Level,analyst data management experience and workin...
4917,data reporting specialist,hra/dept of social services,2024-10-09,aggregate and clean vendor management data fro...,Job types: Full-time\r\nBenefits: Loan forgive...,https://www.indeed.com/m/viewjob?jk=06ebaa320c...,IndeedScraperUSAEng_06ebaa320caf4e24,unknown,ny,manhattan,yearly,95070.0,109330.0,Mid-Level,data reporting specialist aggregate and clean...
2311,data quality engineer java or c,s&p global,2024-11-14,"perform queries to identify, dirty data, orpha...","Benefits: Health insurance, Referral program",https://www.indeed.com/m/viewjob?jk=40e053ff0d...,IndeedScraperUSAEng_40e053ff0d791c1a,unknown,ca,monterey,yearly,80000.0,100000.0,Mid-Level,data quality engineer java or c perform queri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10100,store scanning specialist,retaildata llc,2024-10-29,field representative - flexible schedules for ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=2cf671cb12...,IndeedScraperUSAEng_2cf671cb12cdf3fa,unknown,nc,chapel hill,NaN,NaN,NaN,Mid-Level,store scanning specialist field representative...
2856,data scanning associate,retaildata llc,2024-11-19,field representative - flexible schedules for ...,Job types: Part-time\r\nBenefits: Mileage reim...,https://www.indeed.com/m/viewjob?jk=07d1ef688b...,IndeedScraperUSAEng_07d1ef688b32265d,unknown,ca,oakland,NaN,NaN,NaN,Mid-Level,data scanning associate field representative -...
9539,data scientist,mag aerospace,2024-10-05,presenting information using data visualizatio...,Job types: Full-time\r\nBenefits: Health insur...,https://www.indeed.com/m/viewjob?jk=6fedecb074...,IndeedScraperUSAEng_6fedecb074edd327,unknown,nc,fort liberty,yearly,63750.0,106250.0,Uncategorized,data scientist presenting information using da...
8897,pricing specialist,republic national distributing company,2024-11-10,perform data maintenance: conduct routine data...,"Benefits: Paid training, Paid parental leave, ...",https://www.indeed.com/m/viewjob?jk=e013ee922a...,IndeedScraperUSAEng_e013ee922a7d9926,unknown,tx,grand prairie,NaN,NaN,NaN,Mid-Level,pricing specialist perform data maintenance: c...


In [158]:
chatgpt_rows = chatgpt_rows[['title-blurb', 'job_level']]
print(chatgpt_rows)

                                             title-blurb      job_level
6605   principal full stack engineer database graphql...   Senior-Level
9115   director i data science subject areas our team...   Senior-Level
6641   analyst  data management experience and workin...   Senior-Level
4917   data  reporting specialist aggregate and clean...      Mid-Level
2311   data quality engineer  java or c perform queri...      Mid-Level
...                                                  ...            ...
10100  store scanning specialist field representative...      Mid-Level
2856   data scanning associate field representative -...      Mid-Level
9539   data scientist presenting information using da...  Uncategorized
8897   pricing specialist perform data maintenance: c...      Mid-Level
10046  senior qa architect vast data is looking for a...   Senior-Level

[1400 rows x 2 columns]


In [159]:
#randomly sample 20% of your training data for validation (you never want to use testing set for that)
train_ratio = 0.8 # 80% for training, 20% for validation
random_seed = 100

train_dataframe = chatgpt_rows.sample(frac=train_ratio, random_state=random_seed)
valid_dataframe = chatgpt_rows.drop(train_dataframe.index) #inverse of above
print('training set size:', len(train_dataframe))
print('validation set size:', len(valid_dataframe))

training set size: 1120
validation set size: 280


In [160]:
#get the count of each label
Counter(train_dataframe['job_level'])

Counter({'Senior-Level': 500,
         'Mid-Level': 424,
         'Uncategorized': 112,
         'Entry-Level': 84})

In [161]:
test_dataframe = test_dataframe[['id', 'title-blurb', 'job level']]
test_dataframe

,id,title-blurb,job level
0,IndeedScraperUSAEng_dd69111a392c05c4,senior data engineer performs initial data qua...,Senior-Level
1,IndeedScraperUSAEng_4a21ddda7395ed1a,store scanning specialist flexible part time j...,Entry-Level
2,IndeedScraperUSAEng_61ac4124667518a5,marketing applied scientist ii experience with...,Mid-Level
3,IndeedScraperUSAEng_bbb6a310c0afcc13,research data analyst hybrid opportunity year...,Entry-Level
4,IndeedScraperUSAEng_a4d2ecf9b7c35fa9,bioinformatics data engineer carry out data mo...,Entry-Level
...,...,...,...
594,IndeedScraperUSAEng_b47e13a030427fe1,"director data impact indicators, data sharing...",Mid-Level
595,IndeedScraperUSAEng_f15c91a3428aeac5,senior director data analytics availability ...,Senior-Level
596,IndeedScraperUSAEng_82e0a2aa71e55cf8,principal data scientist drive focused efforts...,Senior-Level
597,IndeedScraperUSAEng_5e43120afd8547ba,data engineer ecommerce experience with perfor...,Mid-Level


In [162]:
#vectorize column
vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2, binary=True) #takes unigrams and bigrams, count more than 0 will be changed to 1.
vectorizer.fit(train_dataframe['title-blurb'])
len(vectorizer.get_feature_names_out()) #all the different possible combinations of 1 and 2 words in the entire dataframe

6148

In [163]:
#just pull the selected training documents out of the matrix
train_X = vectorizer.transform(train_dataframe['title-blurb'])

#just pull selected validation dataframe out of the matrix
valid_X = vectorizer.transform(valid_dataframe['title-blurb'])

#pull out test documents
#test_X = vectorizer.transform(test_dataframe['title-blurb'])
print(train_X.shape)
print(valid_X.shape)
#print(test_X.shape)

(1120, 6148)
(280, 6148)


In [164]:
#convert to training dataframe 
train_y = train_dataframe['job_level'].to_numpy()

#convert validation to df
valid_y = valid_dataframe['job_level'].to_numpy()

#just one column of labels corresponding to documents 
print (train_y.shape)
print (valid_y.shape)

(1120,)
(280,)


In [165]:
num_features_to_select = 6000 #why 5,000? 
feature_selector = SelectKBest(score_func = chi2, k = num_features_to_select) #create chi-square feature selector 
feature_selector.fit(train_X, train_y) #uses training set and results to select the best unigrams and bigrams prediction

# feature names 
all_features = [feature for feature, index in sorted(vectorizer.vocabulary_.items(), key = lambda x: x[1])]


selected_features = feature_selector.get_feature_names_out(input_features = all_features)

In [166]:
train_X_selected = feature_selector.transform(train_X) #reduces to selected features 
valid_X_selected = feature_selector.transform(valid_X)
#test_X_selected = feature_selector.transform(test_X)
print (train_X_selected.shape)
print (valid_X_selected.shape)
#print (test_X_selected.shape)

(1120, 6000)
(280, 6000)


In [180]:
#model = LogisticRegression(C = 1.5, solver='liblinear')
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

model.fit(train_X_selected, train_y)

LogisticRegression(C=1.5, solver='liblinear')

In [181]:
train_y_hat = model.predict(train_X_selected) #predict your labels using these features 
accuracy = accuracy_score(train_y, train_y_hat)
print(accuracy)

1.0


In [182]:
valid_y_hat = model.predict(valid_X_selected)
accuracy = accuracy_score(valid_y, valid_y_hat)
print ('logistic, accuracy on validation set:', accuracy)

sgdclassifier, accuracy on validation set: 0.275


In [183]:
chatgpt_rows

,title-blurb,job_level
6605,principal full stack engineer database graphql...,Senior-Level
9115,director i data science subject areas our team...,Senior-Level
6641,analyst data management experience and workin...,Senior-Level
4917,data reporting specialist aggregate and clean...,Mid-Level
2311,data quality engineer java or c perform queri...,Mid-Level
...,...,...
10100,store scanning specialist field representative...,Mid-Level
2856,data scanning associate field representative -...,Mid-Level
9539,data scientist presenting information using da...,Uncategorized
8897,pricing specialist perform data maintenance: c...,Mid-Level


In [184]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2, binary=True) #takes unigrams and bigrams, count more than 0 will be changed to 1.
vectorizer.fit(chatgpt_rows['title-blurb'])
len(vectorizer.get_feature_names_out()) #all the different possible combinations of 1 and 2 words in the entire dataframe

7441

In [185]:
#just pull the selected training documents out of the matrix
train_X = vectorizer.transform(chatgpt_rows['title-blurb'])

#just pull selected validation dataframe out of the matrix
test_X = vectorizer.transform(test_dataframe['title-blurb'])

#pull out test documents
#test_X = vectorizer.transform(test_dataframe['title-blurb'])
print(train_X.shape)
#print (valid_X.shape)
print (test_X.shape)

(1400, 7441)
(599, 7441)


In [186]:
test_dataframe

,id,title-blurb,job level
0,IndeedScraperUSAEng_dd69111a392c05c4,senior data engineer performs initial data qua...,Senior-Level
1,IndeedScraperUSAEng_4a21ddda7395ed1a,store scanning specialist flexible part time j...,Entry-Level
2,IndeedScraperUSAEng_61ac4124667518a5,marketing applied scientist ii experience with...,Mid-Level
3,IndeedScraperUSAEng_bbb6a310c0afcc13,research data analyst hybrid opportunity year...,Entry-Level
4,IndeedScraperUSAEng_a4d2ecf9b7c35fa9,bioinformatics data engineer carry out data mo...,Entry-Level
...,...,...,...
594,IndeedScraperUSAEng_b47e13a030427fe1,"director data impact indicators, data sharing...",Mid-Level
595,IndeedScraperUSAEng_f15c91a3428aeac5,senior director data analytics availability ...,Senior-Level
596,IndeedScraperUSAEng_82e0a2aa71e55cf8,principal data scientist drive focused efforts...,Senior-Level
597,IndeedScraperUSAEng_5e43120afd8547ba,data engineer ecommerce experience with perfor...,Mid-Level


In [187]:
#convert to training dataframe 
train_y = chatgpt_rows['job_level'].to_numpy()

#convert validation to df
test_y = test_dataframe['job level'].to_numpy()

#just one column of labels corresponding to documents 
print (train_y.shape)
print (test_y.shape)

(1400,)
(599,)


In [188]:
num_features_to_select = 6000 #why 5,000? 
feature_selector = SelectKBest(score_func = chi2, k = num_features_to_select) #create chi-square feature selector 
feature_selector.fit(train_X, train_y) #uses training set and results to select the best unigrams and bigrams prediction

# feature names 
all_features = [feature for feature, index in sorted(vectorizer.vocabulary_.items(), key = lambda x: x[1])]


selected_features = feature_selector.get_feature_names_out(input_features = all_features)

In [189]:
train_X_selected = feature_selector.transform(train_X) #reduces to selected features 
test_X_selected = feature_selector.transform(test_X)
#test_X_selected = feature_selector.transform(test_X)
print (train_X_selected.shape)
print (test_X_selected.shape)
#print (test_X_selected.shape)

(1400, 6000)
(599, 6000)


In [190]:
#model = LogisticRegression(C = 1.5, solver='liblinear')
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

model.fit(train_X_selected, train_y)

LogisticRegression(multi_class='multinomial')

In [191]:
train_y_hat = model.predict(train_X_selected) #predict your labels using these features 
accuracy = accuracy_score(train_y, train_y_hat)
print(accuracy)

1.0


In [193]:
test_y_hat = model.predict(test_X_selected)
accuracy = accuracy_score(test_y, test_y_hat)
print ('logistic regression, accuracy on test set:', accuracy)

logistic regression, accuracy on test set: 0.34223706176961605
